### Model
supplier:
$\begin{aligned} \dot{s}^{S}(\tau) &=o_{r}^{S}\left(\tau-\tau_{2}\right)-o_{r}^{R}\left(\tau-\tau_{1}\right)-b^{S}(\tau) / t_{b} \\ \dot{o}_{u}^{S}(\tau) &=o_{r}^{S}(\tau)-o_{r}^{S}\left(\tau-\tau_{2}\right) \\ \dot{b}^{S}(\tau) &=o_{r}^{R}(\tau)-o_{r}^{R}\left(\tau-\tau_{1}\right)-b^{S}(\tau) / t_{b} \end{aligned}$

retailer: 
$\begin{aligned} \dot{s}^{R}(\tau)&=o_{r}^{R}\left(\tau-\tau_{1}-\tau_{2}\right)+b^{S}\left(\tau-\tau_{2}\right) / t_{b} -d_{r}\left(\tau-\tau_{1}\right)-b^{R}(\tau) / t_{b}\\
\dot{o}_{u}^{R}(\tau)&=o_{r}^{R}(\tau)-o_{r}^{R}\left(\tau-\tau_{1}-\tau_{2}\right)-b^{S}\left(\tau-\tau_{2}\right) / t_{b}\\
\dot{b}^{R}(\tau)&=d_{r}^{R}(\tau)-d_{r}^{R}\left(\tau-\tau_{1}\right)-b^{R}(\tau) / t_{b} \end{aligned}$

#### Pade approximation
delay = 2:$
\begin{array}{l}
-\mathrm{s}^3+6 \mathrm{s}^2-15 \mathrm{s}+15 \\
\hline \mathrm{s}^3 + 6 \mathrm{s}^2+15 \mathrm{s}+15
\end{array}
$

delay = 1: $
\begin{array}{l}
-\mathrm{s}^3+12\mathrm{s}^2-60\mathrm{s}+120 \\
\hline \mathrm{s}^3 + 12\mathrm{s}^2+60 \mathrm{s}+120
\end{array}
$

$\tau_1 = 1$, $\tau_2 = 2$

Transfer into Laplace domain and approximate with 3rd order Pade approximation with

$u_1 = o_r^S$, $u_2 = o_r^R$, $u_3 = d_r^R$

$x_1 = s^S$, $x_2 = o^S_u$, $x_3 = b^S$, $x_4 = z_{11}$, $x_5 = \dot{z}_{11}$, $x_6 = z^{(2)}_{11}$, $x_7 = z_{12}$, $x_8 = \dot{z}_{12}$, $x_9 = z^{2}_{12}$, $x_{10} = s^{R}$, $x_{11} = o_u^R$, $x_{12} = b^R$, $x_{13} = z_{13}$, $x_{14} = \dot{z}_{13}$, $x_{15} = z^{(2)}_{13}$, $x_{16} = z_{14}$, $x_{17} = \dot{z}_{14}$, $x_{18} = z^{(2)}_{14}$, $x_{19} = z_{15}$, $x_{20} = \dot{z}_{15}$, $x_{21} = z^{(2)}_{15}$


$z_{11} = \frac{1}{s^{3} + 12 s^2 + 60 s +120}u_1$, $z_{12} = \frac{1}{s^3 + 6s^2 + 15s +15}u_2$, $z_{13} = \frac{1}{s^3 + 4s^2 + 6.667s + 4.444}u_1$, $z_{14} = \frac{1}{s^{3} + 12 s^2 + 60 s +120}u_2$, $z_{15} = \frac{1}{s^3 + 6s^2 + 15s +15}u_3$

#### State space
$\begin{aligned}\dot{x}_1 &=  -u_1 + 24x_6 + 240x_4 + u_2 - 12x_9 - 30x_7 -x_3/t_b \\\dot{x}_2 &=  2u_1 - 24x_6 - 240x_4\\\dot{x}_3 &=  2u_2 - 12x_9  - 30x_7 -x_3/t_b \\ \dot{x}_{10} &= -u_2 + 8x_{15} + 8.888x_{13} + (-x_3 + 24x_{18} +240x_{16})/t_b  +u_3 -12x_{21} - 30x_{19} - x_{12}/t_b\\ \dot{x}_{11} &= 2u_2 -8x_{15} - 8.888x_{13} -(-x_3 + 24x_{18} + 240x_{16})/t_b\\ \dot{x}_{12} &= 2u_3 -12x_{21} -30x_{19} - x_{12}/t_b        \end{aligned}$

In [1]:
import numpy as np
import casadi as ca
import sympy as sy

In [2]:
def integrator_eular(f, x, u, d, delta_t):
    """
    Explicit Eular solver using casadi.
    Args:
        f: First order ODE in casadi function (Nx + Nt -> Nx).
        t: Current time.
        x: Current value.
        u: Current input.
        delta_t: Step length.
    Returns:
        x_next: Vector of next value in casadi DM
    """
    k1 = f(x, u, d)
    x_next = x + delta_t * k1

    return x_next

In [3]:
Nx = 21
Nu = 3
x_SX = ca.SX.sym("x",Nx)
u_SX = ca.SX.sym("u",Nu)
tb = 1
delta_t = 1

In [4]:
A = ca.DM([
    [0, 0, -1/tb, 240, 0, 24, -30, 0, -12, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 240, 0, 24, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, -1/tb, 0, 0, 0, -30, 0, -12, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, -120, -60, -12, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, -15, -15, -6, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 
    [0, 0, -1/tb, 0, 0, 0, 0, 0, 0, 0, 0, -1/tb, 8.888, 0, 8, 240/tb, 0, 24/tb, -30, 0, -12],    
    [0, 0, 1/tb, 0, 0, 0, 0, 0, 0, 0, 0, 0, -8.888, 0, -8, -240/tb, 0, -24/tb, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1/tb, 0, 0, 0, 0, 0, 0, -30, 0, -12],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -4.444, -6.667, -4, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -120, -60, -12, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -15, -15, -6]
])

In [5]:
A_np = np.array([
    [0, 0, -1/tb, 240, 0, 24, -30, 0, -12, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 240, 0, 24, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, -1/tb, 0, 0, 0, -30, 0, -12, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, -120, -60, -12, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, -15, -15, -6, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 
    [0, 0, -1/tb, 0, 0, 0, 0, 0, 0, 0, 0, -1/tb, 8.888, 0, 8, 240/tb, 0, 24/tb, -30, 0, -12],    
    [0, 0, 1/tb, 0, 0, 0, 0, 0, 0, 0, 0, 0, -8.888, 0, -8, -240/tb, 0, -24/tb, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1/tb, 0, 0, 0, 0, 0, 0, -30, 0, -12],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -4.444, -6.667, -4, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -120, -60, -12, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -15, -15, -6]
])

In [6]:
B = ca.DM([
    [-1,1,0],
    [2,0,0],
    [0,2,0],
    [0,0,0],
    [0,0,0],
    [1,0,0],
    [0,0,0],
    [0,0,0],
    [0,1,0],
    [0,-1,1],
    [0,2,0],
    [0,0,2],
    [0,0,0],
    [0,0,0],
    [0,2,0],
    [0,0,0],
    [0,0,0],
    [0,0,0],
    [0,0,0],
    [0,0,0],
    [0,0,1]
])

In [7]:
B_np = np.array([
    [-1,1,0],
    [2,0,0],
    [0,2,0],
    [0,0,0],
    [0,0,0],
    [1,0,0],
    [0,0,0],
    [0,0,0],
    [0,1,0],
    [0,-1,1],
    [0,2,0],
    [0,0,2],
    [0,0,0],
    [0,0,0],
    [0,2,0],
    [0,0,0],
    [0,0,0],
    [0,0,0],
    [0,0,0],
    [0,0,0],
    [0,0,1]
])

In [8]:
x_dot = A@x_SX + B@ u_SX
x = sy.symbols('x1:22')
u = sy.symbols('u1:4')

x_dot = A_np@x + B_np@ u
x_dot
# s = sy.Symbol('s')

# C_np = np.diag([1] * Nx)
# I = np.diag([1] * Nx)

# G = C_np @ sy.Matrix(s*I -A_np).inv() @ B_np

# G

# G @ sy.Matrix(u)

# A_np_1 = A_np[:9,:9]
# A_np_1

# B_np_1 = B_np[:9,:]
# B_np_1

# x = sy.symbols('x1:10')
# u = sy.symbols('u1:4')
# s = sy.Symbol('s')
# C_np_1 = np.diag([1] * 9)
# I_1 = np.diag([1] * 9)
# G = sy.expand(C_np_1 @ sy.Matrix(s*I -A_np_1).inv() @ B_np_1)
# G

array([-u1 + u2 - 1.0*x3 + 240.0*x4 + 24.0*x6 - 30.0*x7 - 12.0*x9,
       2*u1 + 240.0*x4 + 24.0*x6, 2*u2 - 1.0*x3 - 30.0*x7 - 12.0*x9,
       1.0*x5, 1.0*x6, u1 - 120.0*x4 - 60.0*x5 - 12.0*x6, 1.0*x8, 1.0*x9,
       u2 - 15.0*x7 - 15.0*x8 - 6.0*x9,
       -u2 + u3 - 1.0*x12 + 8.888*x13 + 8.0*x15 + 240.0*x16 + 24.0*x18 - 30.0*x19 - 12.0*x21 - 1.0*x3,
       2*u2 - 8.888*x13 - 8.0*x15 - 240.0*x16 - 24.0*x18 + 1.0*x3,
       2*u3 - 1.0*x12 - 30.0*x19 - 12.0*x21, 1.0*x14, 1.0*x15,
       2*u2 - 4.444*x13 - 6.667*x14 - 4.0*x15, 1.0*x17, 1.0*x18,
       -120.0*x16 - 60.0*x17 - 12.0*x18, 1.0*x20, 1.0*x21,
       u3 - 15.0*x19 - 15.0*x20 - 6.0*x21], dtype=object)

In [9]:
A_sub1 = A_np[0:9,0:9]
A_sub2 = A_np[9:,9:]
B_sub1 = B_np[:9,:2]
B_sub2 = B_np[9:,1:2]
B_sub2 = np.hstack((B_sub2, A_np[9:,2].reshape(-1,1)))

B_pert = B_np[9:,2:]

In [10]:
Nx_sys1 = 9
Nx_sys2 = 12
Nu_sys1 = 2
Nu_sys2 = 2
Nd = 1
x_sys1_SX = ca.SX.sym("x",Nx_sys1)
u_sys1_SX = ca.SX.sym("u",Nu_sys1)

x_sys2_SX = ca.SX.sym("x",Nx_sys2)
u_sys2_SX = ca.SX.sym("u",Nu_sys2)


pert_SX = ca.SX.sym("d",Nd)

In [11]:
f1 = ca.DM(A_sub1) @ x_sys1_SX + ca.DM(B_sub1) @ u_sys1_SX
f1_func = ca.Function("f1_func",[x_sys1_SX,u_sys1_SX,pert_SX],[f1])

f1_dis = integrator_eular(f1_func, x_sys1_SX, u_sys1_SX, pert_SX, delta_t)
f1_dis_func = ca.Function("f1_dis_func",[x_sys1_SX,u_sys1_SX,pert_SX],[f1_dis])

In [12]:
f2 = ca.DM(A_sub2) @ x_sys2_SX + ca.DM(B_sub2) @ u_sys2_SX + ca.DM(B_pert) @ pert_SX
f2_func = ca.Function("f2_func",[x_sys2_SX,u_sys2_SX,pert_SX],[f2])

f2_dis = integrator_eular(f2_func, x_sys2_SX, u_sys2_SX, pert_SX, delta_t)
f2_dis_func = ca.Function("f2_dis_func",[x_sys2_SX,u_sys2_SX,pert_SX],[f2_dis])

f_list= [f1_dis, f2_dis]

$$
\begin{array}{l}
N=6, W_{s}^{R}=30, W_{o}^{R}=30, W_{\Delta}^{R}=1 \\
W_{s}^{S}=30, W_{o}^{S}=30, W_{\Delta}^{S}=1
\end{array}
$$

#### Cost function: $$
\begin{aligned}
J_{1} &=\sum_{k=0}^{N-1}\left(r_{s, k}^{R}-s_{k}^{R}\right)^{2} W_{s}^{R}+\left(r_{o, k}^{R}-o_{k}^{R}\right)^{2} W_{o}^{R}+\left(\Delta o_{k}^{R}\right)^{2} W_{\Delta}^{R} \\
J_{2} &=\sum_{k=0}^{N-1}\left(r_{s, k}^{S}-s_{k}^{S}\right)^{2} W_{s}^{S}+\left(r_{o, k}^{S}-o_{k}^{S}\right)^{2} W_{o}^{S}+\left(\Delta o_{k}^{S}\right)^{2} W_{\Delta}^{S}
\end{aligned}
$$

In [13]:
def create_opt_problem(fi_func, N_xi, hi_func):
    xi = ca.SX.sym("xi",N_xi)
    fi = fi_func(xi)
    g = hi_func(xi)
    # Define proximal solver
    solver_opt = {}
    solver_opt['print_time'] = False
    solver_opt['ipopt'] = {
        'max_iter': 500,
        'print_level': 1,
        'acceptable_tol': 1e-6,
        'acceptable_obj_change_tol': 1e-6
    }

    nlp = {'x':xi, 'g':g, 'f':fi}
#     print(nlp)
    solver = ca.nlpsol('solver', 'ipopt', nlp, solver_opt)
    return solver

In [14]:
def create_subproblem(fi_func, Ai, rho, hi_func, para_i, D_SX):
    N_lambda, N_yi = np.shape(Ai)
    yi = ca.SX.sym("yi",N_yi)
    xi = ca.SX.sym("xi",N_yi)
    sigma_i = ca.SX.sym('sigma_i',N_yi,N_yi)
    lambda_ = ca.SX.sym("lambda",N_lambda)
    
    fi = fi_func(yi, para_i) + lambda_.T @ Ai @ yi + rho/2 * (yi - xi).T @ sigma_i @ (yi - xi)
    print(np.shape(lambda_),np.shape(xi),np.shape(ca.reshape(sigma_i, -1,1)),np.shape(para_i),np.shape(ca.reshape(D_SX,-1,1)))
    
    p = ca.vertcat(lambda_, xi, ca.reshape(sigma_i, -1,1), para_i, ca.reshape(D_SX,-1,1))
    g = hi_func(yi, D_SX)
    # Define proximal solver
    solver_opt = {}
    solver_opt['print_time'] = False
    solver_opt['ipopt'] = {
        'max_iter': 500,
        'print_level': 1,
        'acceptable_tol': 1e-6,
        'acceptable_obj_change_tol': 1e-6
    }

    nlp = {'x':yi, 'g':g, 'f':fi, 'p': p}
#     print(nlp)
    solver = ca.nlpsol('solver', 'ipopt', nlp, solver_opt)
    return solver

In [15]:
def line_search(penalty_func, x, delta_y, lambda_bar, kappa_bar, para, D):
    alpha = 1
    while( penalty_func(x,lambda_bar,kappa_bar, para, D) < penalty_func(x + alpha * delta_y, lambda_bar,kappa_bar, para, D)):
#         print(penalty_func(x,lambda_bar,kappa_bar),penalty_func(x + alpha * delta_y, lambda_bar,kappa_bar) )
        alpha = alpha * 0.8
        if alpha < 1e-9:
            break
    return alpha

In [16]:
def constraint_jac_approx(yi, hi_func, hi_jac_func, D_SX):
    constraint_res = hi_func(yi, D_SX)    #  Residue
    Nh = np.shape(constraint_res)[0]
    Ny = np.shape(yi)[0]
    zero_row = ca.DM.zeros(1,Ny)
    hi_jac = hi_jac_func(yi, D_SX)
    for i in range(Nh):
        if constraint_res[i] != 0:    #  TODO: deal with small value
            hi_jac[i,:] = zero_row
    hi_jac = ca.DM.zeros(Nh,Ny)
    return hi_jac

In [17]:
def modified_grad(fi_grad, hi_jac_approx, hi_jac_real, kappa_i):
    return fi_grad + (hi_jac_real - hi_jac_approx).T @ kappa_i

In [18]:
def create_QP_problem(A_list, b,  mu, N_hi_list):
    N = len(A_list)
    N_lambda = np.shape(A_list[0])[0]
    
    s = ca.SX.sym("s", N_lambda)
    lambda_ = ca.SX.sym("lambda_", N_lambda)
    
    delta_yi_list = []
    fkh_hess_col_list = [] 
    modiefied_grad_col_list = []
    Ci_col_list = []
    
    yi_list = []
    obj = 0
    sigma_Ai = 0
    g = []
    for i in range(N):
        Ai = A_list[i]
        N_delta_yi = np.shape(Ai)[1]
        Hi = ca.SX.sym("Hi" + str(i), N_delta_yi, N_delta_yi)
        gi = ca.SX.sym("gi" + str(i), N_delta_yi)
        yi = ca.SX.sym("yi" + str(i), N_delta_yi)
        Ci = ca.SX.sym("Ci" + str(i), N_hi_list[i], N_delta_yi)
        
        fkh_hess_col_list += [ca.reshape(Hi, -1, 1)]
        modiefied_grad_col_list += [ca.reshape(gi, -1, 1)]
        yi_list += [yi]
        
        delta_yi = ca.SX.sym("delta_yi" + str(i),N_delta_yi)
        delta_yi_list += [delta_yi]
    
        obj += 1/2 * delta_yi.T @ Hi @ delta_yi + gi.T @ delta_yi
        sigma_Ai += Ai @ (yi + delta_yi)
        
        Ci_col_list += [ca.reshape(Ci, -1, 1)]
        g += [Ci @ delta_yi]
    obj += lambda_.T @ s + mu/2 * s.T @ s
    x = ca.vertcat(*delta_yi_list, s)
    p = ca.vertcat(lambda_, *(fkh_hess_col_list + modiefied_grad_col_list + yi_list + Ci_col_list))

    g += [ sigma_Ai - b - s ]
    g = ca.vertcat(*g)
    # Define proximal solver
    solver_opt = {}
    solver_opt['print_time'] = False
    solver_opt['ipopt'] = {
        'max_iter': 500,
        'print_level': 1,
        'acceptable_tol': 1e-10,
        'acceptable_obj_change_tol': 1e-10
    }

    nlp = {'x':x, 'g':g, 'f':obj, 'p': p}
#     print(nlp)
    solver = ca.nlpsol('solver', 'ipopt', nlp, solver_opt)
    return solver    

In [19]:
def create_penalty_func(fi_func_list, A_list, b, hi_func_list, para_list, D_SX):
    N = len(A_list)
    
    lambda_bar = ca.SX.sym("lambda_bar", 1)
    kappa_bar = ca.SX.sym("kappa_bar", 1)
    
    xi_list = []
    para_i_list = []
    Phi = 0
    sum_Ax = 0
    for i in range(N):
        Ai = A_list[i]
        N_xi = np.shape(Ai)[1]
        xi = ca.SX.sym("xi" + str(i), N_xi)
        xi_list += [xi]
        
        para_i = para_list[i]
        para_i_list += [para_i]
        fi_func = fi_func_list[i]
        Phi += fi_func(xi,para_i)
        
        hi_func = hi_func_list[i]
        hi = hi_func(xi, D_SX)
        N_hi = np.shape(hi)[0]
        for j in range(N_hi):
            Phi += kappa_bar * ca.fmax(0, hi[j])
        sum_Ax += Ai @ xi

    Phi += lambda_bar * ca.norm_1(sum_Ax - b) 
    x = ca.vertcat(*xi_list)
    para = ca.vertcat(*para_i_list)
    penalty_func = ca.Function("penalty_func", [x, lambda_bar, kappa_bar, para, D_SX ], [Phi])
    return penalty_func

In [20]:
N = 6
rho = 1
lambda_bar = 1
kappa_bar = 1e4

ref_sS = ca.SX.sym("ref_stock_supplier",N)
ref_sR = ca.SX.sym("ref_stock_retailer",N)
ref_oS = ca.SX.sym("ref_order_supplier",N)
ref_oR = ca.SX.sym("ref_order_retailer",N)


tb = 1
delta_t = 1


W_sR = 30
W_oR = 30

W_deltaR = 1
W_deltaS = 1

W_sS = 30
W_oS = 30

In [21]:
# Nx_sys1 = 9
# Nx_sys2 = 12
# Nu_sys1 = 2
# Nu_sys2 = 2

# system 1: Supply
X_sys1_SX = ca.SX.sym("X1_SX", Nx_sys1, N)
U_sys1_SX = ca.SX.sym("U1_SX", Nu_sys1, N)
o_rS_para = ca.SX.sym("o_rS_past", 1)
# system 2: Retailer
X_sys2_SX = ca.SX.sym("X2_SX", Nx_sys2, N)
U_sys2_SX = ca.SX.sym("U2_SX", Nu_sys2, N)
o_rR_para = ca.SX.sym("o_rR_past", 1)

D_SX = ca.SX.sym("D_SX", Nd, N)

decision_var_sys1 = ca.vertcat(ca.reshape(X_sys1_SX,-1,1), ca.reshape(U_sys1_SX,-1,1))
decision_var_sys2 = ca.vertcat(ca.reshape(X_sys2_SX,-1,1), ca.reshape(U_sys2_SX,-1,1))

In [22]:
# system 1: Supply
s_S = X_sys1_SX[0,:].T
o_S = X_sys1_SX[1,:].T
# system 2: Retailer
s_R = X_sys2_SX[0,:].T
o_R = X_sys2_SX[1,:].T

In [23]:
obj1 =  0
obj2 =  0

obj1 += W_sS * (ref_sS - s_S).T @ (ref_sS - s_S) + W_oS * (ref_oS - o_S).T @ (ref_oS - o_S)
obj2 += W_sR * (ref_sR - s_R).T @ (ref_sR - s_R) + W_oR * (ref_oR - o_R).T @ (ref_oR - o_R)

o_rS_past = o_rS_para
o_rR_past = o_rR_para


for i in range(N):
    o_rS_k = U_sys1_SX[0,i]
    o_rR_k = U_sys2_SX[0,i]
    
    obj1 +=  W_deltaS * (o_rS_past - o_rS_k) ** 2
    obj2 +=  W_deltaR * (o_rR_past - o_rR_k) ** 2
    
    o_rS_past = o_rS_k
    o_rR_past = o_rR_k

para_S = ca.vertcat(ref_sS, ref_oS, o_rS_para)
para_R = ca.vertcat(ref_sR, ref_oR, o_rR_para)

para_list = [para_S, para_R]

obj_list = [obj1, obj2]

obj1_func = ca.Function("obj1_func", [decision_var_sys1, para_S], [obj1])
obj2_func = ca.Function("obj2_func", [decision_var_sys2, para_R], [obj2])

fi_list = []
fi_list += [obj1]
fi_list += [obj2]

fi_func_list = []
fi_func_list += [obj1_func]
fi_func_list += [obj2_func]

#### Define the ALADIN problem

In each prediction step of N: 

u2_sys1 = u1_sys2

x3_sys1 = u2_sys2

In [24]:
decision_var_sys1

SX([X1_SX_0, X1_SX_1, X1_SX_2, X1_SX_3, X1_SX_4, X1_SX_5, X1_SX_6, X1_SX_7, X1_SX_8, X1_SX_9, X1_SX_10, X1_SX_11, X1_SX_12, X1_SX_13, X1_SX_14, X1_SX_15, X1_SX_16, X1_SX_17, X1_SX_18, X1_SX_19, X1_SX_20, X1_SX_21, X1_SX_22, X1_SX_23, X1_SX_24, X1_SX_25, X1_SX_26, X1_SX_27, X1_SX_28, X1_SX_29, X1_SX_30, X1_SX_31, X1_SX_32, X1_SX_33, X1_SX_34, X1_SX_35, X1_SX_36, X1_SX_37, X1_SX_38, X1_SX_39, X1_SX_40, X1_SX_41, X1_SX_42, X1_SX_43, X1_SX_44, X1_SX_45, X1_SX_46, X1_SX_47, X1_SX_48, X1_SX_49, X1_SX_50, X1_SX_51, X1_SX_52, X1_SX_53, U1_SX_0, U1_SX_1, U1_SX_2, U1_SX_3, U1_SX_4, U1_SX_5, U1_SX_6, U1_SX_7, U1_SX_8, U1_SX_9, U1_SX_10, U1_SX_11])

In [25]:
decision_var_sys2

SX([X2_SX_0, X2_SX_1, X2_SX_2, X2_SX_3, X2_SX_4, X2_SX_5, X2_SX_6, X2_SX_7, X2_SX_8, X2_SX_9, X2_SX_10, X2_SX_11, X2_SX_12, X2_SX_13, X2_SX_14, X2_SX_15, X2_SX_16, X2_SX_17, X2_SX_18, X2_SX_19, X2_SX_20, X2_SX_21, X2_SX_22, X2_SX_23, X2_SX_24, X2_SX_25, X2_SX_26, X2_SX_27, X2_SX_28, X2_SX_29, X2_SX_30, X2_SX_31, X2_SX_32, X2_SX_33, X2_SX_34, X2_SX_35, X2_SX_36, X2_SX_37, X2_SX_38, X2_SX_39, X2_SX_40, X2_SX_41, X2_SX_42, X2_SX_43, X2_SX_44, X2_SX_45, X2_SX_46, X2_SX_47, X2_SX_48, X2_SX_49, X2_SX_50, X2_SX_51, X2_SX_52, X2_SX_53, X2_SX_54, X2_SX_55, X2_SX_56, X2_SX_57, X2_SX_58, X2_SX_59, X2_SX_60, X2_SX_61, X2_SX_62, X2_SX_63, X2_SX_64, X2_SX_65, X2_SX_66, X2_SX_67, X2_SX_68, X2_SX_69, X2_SX_70, X2_SX_71, U2_SX_0, U2_SX_1, U2_SX_2, U2_SX_3, U2_SX_4, U2_SX_5, U2_SX_6, U2_SX_7, U2_SX_8, U2_SX_9, U2_SX_10, U2_SX_11])

In [26]:
X_sys1_SX

SX(
[[X1_SX_0, X1_SX_9, X1_SX_18, X1_SX_27, X1_SX_36, X1_SX_45], 
 [X1_SX_1, X1_SX_10, X1_SX_19, X1_SX_28, X1_SX_37, X1_SX_46], 
 [X1_SX_2, X1_SX_11, X1_SX_20, X1_SX_29, X1_SX_38, X1_SX_47], 
 [X1_SX_3, X1_SX_12, X1_SX_21, X1_SX_30, X1_SX_39, X1_SX_48], 
 [X1_SX_4, X1_SX_13, X1_SX_22, X1_SX_31, X1_SX_40, X1_SX_49], 
 [X1_SX_5, X1_SX_14, X1_SX_23, X1_SX_32, X1_SX_41, X1_SX_50], 
 [X1_SX_6, X1_SX_15, X1_SX_24, X1_SX_33, X1_SX_42, X1_SX_51], 
 [X1_SX_7, X1_SX_16, X1_SX_25, X1_SX_34, X1_SX_43, X1_SX_52], 
 [X1_SX_8, X1_SX_17, X1_SX_26, X1_SX_35, X1_SX_44, X1_SX_53]])

In [27]:
U_sys1_SX

SX(
[[U1_SX_0, U1_SX_2, U1_SX_4, U1_SX_6, U1_SX_8, U1_SX_10], 
 [U1_SX_1, U1_SX_3, U1_SX_5, U1_SX_7, U1_SX_9, U1_SX_11]])

In [28]:
(Nx_sys1 + Nu_sys1) * N 

66

In [29]:
decision_var_sys1[Nx_sys1 * N + Nu_sys1 * 0 + 2]

SX(U1_SX_2)

In [30]:
# Select matrix A to statisfy the coupled constraints
# 2 x N row for each Ai

A1 = ca.DM.zeros(2 * N , (Nx_sys1 + Nu_sys1) * N )
A2 = ca.DM.zeros(2 * N , (Nx_sys2 + Nu_sys2) * N )

for i in range(N):
    A1[i, Nx_sys1 * N + Nu_sys1 * i + 1] = 1
    A1[i + N, 2 + Nx_sys1 * i ] = 1
    
    A2[i, Nx_sys2 * N + Nu_sys2 * i ] = -1
    A2[i + N, Nx_sys2 * N + Nu_sys2 * i +  1] = -1

In [31]:
A1 @ decision_var_sys1 + A2 @ decision_var_sys2

SX([(U1_SX_1-U2_SX_0), (U1_SX_3-U2_SX_2), (U1_SX_5-U2_SX_4), (U1_SX_7-U2_SX_6), (U1_SX_9-U2_SX_8), (U1_SX_11-U2_SX_10), (X1_SX_2-U2_SX_1), (X1_SX_11-U2_SX_3), (X1_SX_20-U2_SX_5), (X1_SX_29-U2_SX_7), (X1_SX_38-U2_SX_9), (X1_SX_47-U2_SX_11)])

In [32]:
np.random.seed(1)
eps = 1e-5

N_itermax = 10

# Define A matrix
A_list = []
A_list += [A1]
A_list += [A2]
# Define b
b = ca.DM.zeros(2 * N,1)

In [33]:
obj1

SX(@1=30, ((((((((((((((@1*(ref_stock_supplier_0-X1_SX_0))*(ref_stock_supplier_0-X1_SX_0))+((@1*(ref_stock_supplier_1-X1_SX_9))*(ref_stock_supplier_1-X1_SX_9)))+((@1*(ref_stock_supplier_2-X1_SX_18))*(ref_stock_supplier_2-X1_SX_18)))+((@1*(ref_stock_supplier_3-X1_SX_27))*(ref_stock_supplier_3-X1_SX_27)))+((@1*(ref_stock_supplier_4-X1_SX_36))*(ref_stock_supplier_4-X1_SX_36)))+((@1*(ref_stock_supplier_5-X1_SX_45))*(ref_stock_supplier_5-X1_SX_45)))+(((((((@1*(ref_order_supplier_0-X1_SX_1))*(ref_order_supplier_0-X1_SX_1))+((@1*(ref_order_supplier_1-X1_SX_10))*(ref_order_supplier_1-X1_SX_10)))+((@1*(ref_order_supplier_2-X1_SX_19))*(ref_order_supplier_2-X1_SX_19)))+((@1*(ref_order_supplier_3-X1_SX_28))*(ref_order_supplier_3-X1_SX_28)))+((@1*(ref_order_supplier_4-X1_SX_37))*(ref_order_supplier_4-X1_SX_37)))+((@1*(ref_order_supplier_5-X1_SX_46))*(ref_order_supplier_5-X1_SX_46))))+sq((o_rS_past-U1_SX_0)))+sq((U1_SX_0-U1_SX_2)))+sq((U1_SX_2-U1_SX_4)))+sq((U1_SX_4-U1_SX_6)))+sq((U1_SX_6-U1_SX_8)))

In [34]:
# Define gradtient function
fi_grad_list = []
fi_grad_func_list = []

fi_grad1 = ca.gradient(fi_list[0], decision_var_sys1)
fi_grad2 = ca.gradient(fi_list[1], decision_var_sys2)
fi_grad_list += [fi_grad1]
fi_grad_list += [fi_grad2]

fi_grad_func1 = ca.Function("fi_grad_func1", [decision_var_sys1, para_S], [fi_grad1])
fi_grad_func2 = ca.Function("fi_grad_func2", [decision_var_sys2, para_R], [fi_grad2])
fi_grad_func_list += [fi_grad_func1]
fi_grad_func_list += [fi_grad_func2]

In [35]:
# Define inequality constraints
h1i_list = []
h2i_list = []

# h1i_list += [X_sys1_SX[:,0]]
# h2i_list += [X_sys2_SX[:,0]]


hi_func_list = []
Nh_list = []
for i in range(N - 1):
    x1_k = X_sys1_SX[:,i]
    u1_k = U_sys1_SX[:,i]
    dk = D_SX[:,i]
    x2_k = X_sys2_SX[:,i]
    u2_k = U_sys2_SX[:,i]
    
    x1_next = X_sys1_SX[:,i+1]
    x2_next = X_sys2_SX[:,i+1]    
    
    h1i = f1_dis_func(x1_k,u1_k,dk) - x1_next
    h2i = f2_dis_func(x2_k,u2_k,dk) - x2_next

    h1i_list += [h1i]
    h2i_list += [h2i]    
h1 = ca.vertcat(*h1i_list)
h2 = ca.vertcat(*h2i_list)
h_list = [h1,h2]
h1i_func = ca.Function("h1i_func", [decision_var_sys1, D_SX], [h1])
h2i_func = ca.Function("h2i_func", [decision_var_sys2, D_SX], [h2])
hi_func_list += [h1i_func]
hi_func_list += [h2i_func]
# Deal with the number of inequality constraints for each i.
Nh1 = np.shape(h1)[0]
Nh2 = np.shape(h2)[0]
Nh_list += [Nh1,Nh2]

In [36]:
# Define approximate jacobian, real jacobian and Hessian.
kappa_list = []
h_jac_list = []
h_jac_func_list = []
fkh_hess_list = []
fkh_hess_func_list = []

# Kappa
kappa1 = ca.SX.sym("kappa1",Nh_list[0])
kappa2 = ca.SX.sym("kappa2",Nh_list[1])
kappa_list += [kappa1]
kappa_list += [kappa2]
# Jacobian function
h1_jac = ca.jacobian(h_list[0],decision_var_sys1)
h2_jac = ca.jacobian(h_list[1],decision_var_sys2)
h_jac_list += [h1_jac]
h_jac_list += [h2_jac]

h1_jac_func = ca.Function("h1_jac_func",[decision_var_sys1, D_SX],[h1_jac])
h2_jac_func = ca.Function("h2_jac_func",[decision_var_sys2, D_SX],[h2_jac])
h_jac_func_list +=  [h1_jac_func]
h_jac_func_list +=  [h2_jac_func]
# Hessian fucntion
fkh_1 = fi_list[0] + kappa1.T @ h_list[0]
fkh_2 = fi_list[1] + kappa2.T @ h_list[1]
fkh_hess_1 = ca.hessian(fkh_1, decision_var_sys1)[0]
fkh_hess_2 = ca.hessian(fkh_2, decision_var_sys2)[0]
fkh_hess_1_func = ca.Function("fkh_hess_1_func", [decision_var_sys1, D_SX, kappa1], [fkh_hess_1])
fkh_hess_2_func = ca.Function("fkh_hess_2_func", [decision_var_sys2, D_SX, kappa2], [fkh_hess_2])
fkh_hess_list += [fkh_hess_1]
fkh_hess_list += [fkh_hess_2]
fkh_hess_func_list += [fkh_hess_1_func]
fkh_hess_func_list += [fkh_hess_2_func]

In [37]:
penalty_func = create_penalty_func(fi_func_list, A_list, b, hi_func_list, para_list, D_SX)

In [38]:
subsolver_list = []
# Define subproblem solvers
for i in range(2):
    Ai = A_list[i]
    fi_func = fi_func_list[i]
    hi_func = hi_func_list[i]
    para_i = para_list[i]
    subsolver_list += [create_subproblem(fi_func, Ai, rho, hi_func, para_i, D_SX)]

(12, 1) (66, 1) (4356, 1) (13, 1) (6, 1)
(12, 1) (84, 1) (7056, 1) (13, 1) (6, 1)


In [39]:
# 12 66 4356 3 6

In [40]:
# Define QP problem
mu = 1
QP_solver = create_QP_problem(A_list, b,  mu, Nh_list)

In [41]:
# Initial guess
delta_yi_list = []
Sigma_i_list = []
xi_list = []
yi_list = []
lbhi_list = []
ubhi_list = []
lbx_list = []
ubx_list = []
Nx = 0
Nhi_sum = 0
for i in range(2):
    Ai = A_list[i]
    N_lambda, N_xi = np.shape(Ai)
    Nx += N_xi
    Nhi = Nh_list[i]
    Nhi_sum += Nhi
    xi = np.random.randn(N_xi,1).flatten().tolist()
    

# #     xi = ca.DM.zeros(N_xi,1).full().flatten().tolist()
#     if i == N-1:
#         xi = ca.vertcat(eta_list[N-1],eta_list[0]).full().flatten().tolist()
#     else:
#         xi = ca.vertcat(eta_list[i],eta_list[i+1]).full().flatten().tolist()
    xi_list += [xi]

#     print(N_xi)
    Sigma_i_list += [ca.diag([1.5] * N_xi)]
    
    lbhi_list += [[0] * Nhi]
    ubhi_list += [[0] * Nhi]
    yi = np.random.randn(N_xi,1).flatten().tolist()
#     yi = xi
    yi_list += [yi]   
#     yi_list += [[0] * N_xi]
    lbx_temp = [-ca.inf] * N_xi
    ubx_temp = [ca.inf] * N_xi
    
    if i == 0:
        lbx_temp[0] = 250
        ubx_temp[0] = 250
    if i == 1:
        lbx_temp[0] = 250
        ubx_temp[0] = 250
    lbx_list += [lbx_temp]
    ubx_list += [ubx_temp]
lambda_ = np.random.randn(N_lambda,1)
lambda_ = ca.DM(lambda_)
# lambda_ = ca.DM.zeros(N_lambda,1)
s_list = [0] * N_lambda
delta_yi_list = sum(yi_list,[])

In [42]:
# Define solver
nl_sub = {}

nl_qp = {}
nl_qp['lbg'] = [0] * (Nhi_sum + N_lambda)
nl_qp['ubg'] = [0] * (Nhi_sum + N_lambda)
nl_qp['lbx'] = sum(lbx_list,[]) + [-np.inf] * N_lambda    # delta_y and s lower bound
nl_qp['ubx'] = sum(ubx_list,[]) + [+np.inf] * N_lambda    # delta_y and s upper boun

In [43]:
# Track solution
yi_sol_list = []
delta_y_sol_list = []
lambda_list = []
x_sol_list = []
s_sol_list = []
x_sol_list += [xi_list.copy()]
lambda_list += lambda_.full().flatten().tolist()

In [44]:
para_val_list = [[230] *N + [50] * N + [50], [150] * N + [50] *N + [50]]# Supplier [230, 50, 50], Retailer: 
para_val_DM = ca.DM([230] *N + [50] * N + [50] + [150] * N + [50] *N + [50])
D_list = [70] * 6

In [45]:
len(para_val_list)

2

In [46]:
# solve problem



for i in range(N_itermax):
    sum_Ay = 0
    kappa_sol_list = []
    # Step 1: solve the subproblem      
    for j in range(2):
        Ai = A_list[j]
        N_lambda_i, N_xi = np.shape(Ai)
        Sigma_i = Sigma_i_list[j]
        nl_sub['lbg'] = lbhi_list[j]
        nl_sub['ubg'] = ubhi_list[j]    
        nl_sub['x0'] = yi_list[j]
        nl_sub['lbx'] = lbx_list[j]
        nl_sub['ubx'] = ubx_list[j]
#         print(len(lambda_list),len(xi_list[j]),len(ca.reshape(Sigma_i, -1, 1).full().flatten().tolist()), len(para_val_list[j]),len(D_list))
        nl_sub['p'] = lambda_list + xi_list[j] + ca.reshape(Sigma_i, -1, 1).full().flatten().tolist() + para_val_list[j] + D_list
#         print(nl_sub)
        solver_subproblem = subsolver_list[j]
        yi_sol = solver_subproblem(**nl_sub)
#         print(yi_sol)
        yi_list[j] = yi_sol['x'].full().flatten().tolist()
#         print("yi_sol",yi_sol['x'])
        yi_sol_list += [yi_list[j].copy()]
        kappa_i_sol = yi_sol['lam_g']
        kappa_sol_list += [kappa_i_sol]
        
        sum_Ay += Ai @ yi_sol['x']

    # Step 2: Check if the tolerance satisfied
    #TODO: modify
#     N_flag = 0
#     for j in range(N):
#         if rho * ca.norm_1( Sigma_i_list[j] @ ca.DM(yi_list[j])) <= eps:
#             N_flag += 1
#     if ca.norm_1(sum_Ay - b) <= eps and N_flag == N:
#         break
    # Step3: update Jacobian approximations, calculate the modified gradient, and update Hessian
    Ci_list = []    #  constraint Jacobian
    g_list = []    #  modified gradient
    H_list = []    #  Hessian
    for j in range(2):
        # 3.1 Choose Jacobian approximations
        yi = yi_list[j]
        hi_func = hi_func_list[j]
        hi_jac_func = h_jac_func_list[j]
        fkh_hess_func = fkh_hess_func_list[j]
        hi = hi_func(yi, ca.DM(D_list))
        kappa_i_sol = kappa_sol_list[j]
        fi_grad = fi_grad_func_list[j](yi, ca.DM(para_val_list[j]))
        hi_jac_real = hi_jac_func(yi, ca.DM(D_list))
        
        hi_jac_approx = constraint_jac_approx(yi, hi_func, hi_jac_func, ca.DM(D_list))
        Ci_list += [ca.reshape(hi_jac_approx, -1, 1)]
        gi = modified_grad(fi_grad, hi_jac_approx, hi_jac_real, kappa_i_sol)
        g_list += [ca.reshape(gi, -1, 1)]
        
        Hi = fkh_hess_func(yi, ca.DM(D_list),kappa_i_sol)
        H_list += [ca.reshape(Hi, -1, 1)]
#         print("hi", hi, "kappa_i_sol",kappa_i_sol, "fi_grad",fi_grad, "hi_jac_real",hi_jac_real, "hi_jac_approx",hi_jac_approx, "gi",gi, "Hi" ,Hi)
    # Step 4: Solve QP problem
    nl_qp['x0'] = delta_yi_list + s_list    #  Initial guess
    
    H_para = ca.vertcat(*H_list)
    modified_grad_value = ca.vertcat(*g_list)
    y = ca.vertcat(* sum(yi_list,[]))
    Ci = ca.vertcat(*Ci_list)
    lambda_ = ca.vertcat(lambda_list)
    p = ca.vertcat(lambda_, H_para, modified_grad_value, y, Ci)
    nl_qp['p'] = ca.DM(p)
#     print("lambda_",lambda_)
#     print("H_para", H_para)
#     print("modified_grad_value", modified_grad_value)
#     print("y",y)
#     print("Ci",Ci)
    QP_sol = QP_solver(**nl_qp)
#     print(QP_sol)
    alpha1 = 1
    alpha2 = 1
    alpha3 = 1
    # Step 5: Update x and lambda
    pos = 0
#     print(QP_sol['x'][0:Nx,:])
    delta_y = QP_sol['x'][0:Nx,:]
    delta_y_sol_list += [delta_y]
    s_sol = QP_sol['x'][Nx:,:]
    s_sol_list += [s_sol]
    QP_list = QP_sol['x'].full().flatten().tolist()
    lambda_QP = QP_sol['lam_g'][:N_lambda]
#     print("lambda_QP", lambda_QP)
    x = ca.DM(sum(xi_list,[]))
    y = ca.DM(sum(yi_list,[]))
#     print(delta_y)
#     print(y,x)
    alpha_update = line_search(penalty_func, x, delta_y, lambda_bar, kappa_bar * 1.1,  para_val_DM, ca.DM(D_list))
    print(alpha_update)
    x_plus = x + alpha1 * (y - x) + alpha_update * delta_y
#     x_plus = x + alpha1 * (y - x) + alpha2 * delta_y
    for j in range(2):
        list_len = len(xi_list[j])
        xi_list[j] = x_plus[pos:pos+list_len].full().flatten().tolist()
        pos = pos+list_len
#     print(xi_list)
#     print(lambda_QP)
    lambda_ = lambda_ + alpha_update * (lambda_QP - lambda_)
#     lambda_ = lambda_ + alpha3 * (lambda_QP - lambda_)
    lambda_list = lambda_.full().flatten().tolist()
    x_sol_list += [xi_list.copy()]


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

9.713344461128698e-10
9.713344461128698e-10
1.517710072051359e-09
2.3714219875802478e-09
4.6316835694926705e-09
7.237005577332297e-09
1.4134776518227139e-08
2.2085588309729903e-08
1.3164036458569703e-07
2.056880696651516e-07


In [47]:
x_sol_list

[[[1.6243453636632417,
   -0.6117564136500754,
   -0.5281717522634557,
   -1.0729686221561705,
   0.8654076293246785,
   -2.3015386968802827,
   1.74481176421648,
   -0.7612069008951028,
   0.31903909605709857,
   -0.2493703754774101,
   1.462107937044974,
   -2.060140709497654,
   -0.3224172040135075,
   -0.38405435466841564,
   1.1337694423354374,
   -1.0998912673140309,
   -0.17242820755043575,
   -0.8778584179213718,
   0.04221374671559283,
   0.5828152137158222,
   -1.1006191772129212,
   1.1447237098396141,
   0.9015907205927955,
   0.5024943389018682,
   0.9008559492644118,
   -0.6837278591743331,
   -0.12289022551864817,
   -0.9357694342590688,
   -0.2678880796260159,
   0.530355466738186,
   -0.691660751725309,
   -0.39675352685597737,
   -0.6871727001195994,
   -0.8452056414987196,
   -0.671246130836819,
   -0.01266459891890136,
   -1.1173103486352778,
   0.23441569781709215,
   1.6598021771098705,
   0.7420441605773356,
   -0.19183555236161492,
   -0.8876289640848363,
   -0.